# Keras layered model: CNN

# The Script Contents 

1. Importing Libraries and Data
2. Data Wrangling
3. Reshaping for modeling
4. Data Split
5. Bayesian Hyper-parameter Optimisation
6. Creating Keras Model with Optimised Hyper-parameters¶
7. Creating Confusion Matrix 

# 1. Importing Libraries and Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import time
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.multiclass import type_of_target
from keras.layers import Conv1D, Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from keras.layers.core import Dense, Dropout
from keras.utils import np_utils
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

2024-11-25 14:28:19.986781: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
path = r'/home/Tim/Downloads'

In [3]:
df = pd.read_csv(os.path.join(path, 'Original Data X_cleaned.csv'))

In [4]:
df1 = pd.read_csv(os.path.join(path, 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))

In [5]:
df.head()

,DATE,MONTH,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_cloud_cover,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_pressure,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_humidity,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.00,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,0.83,1.0161,0.12,0.08,0.0,10.0,7.0,11.5,7,0.91,1.0010,0.13,0.22,0.0,10.6,9.4,8.3,8,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.92,1.0260,0.53,0.0,1.4,7.6,4.4,10.8,5,0.67,1.018,0.20,0.10,0.0,6.9,1.1,10.4,8,0.98,0.9978,0.04,1.14,0.0,4.9,3.8,5.9,4,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,0.98,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,19600102,1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,0.84,1.0172,0.25,0.00,0.0,2.9,2.2,4.4,4,0.67,1.017,0.18,0.31,0.0,2.3,1.4,3.1,8,0.90,1.0056,0.14,0.06,0.1,7.7,6.4,8.3,8,0.89,1.0161,0.18,0.66,0.5,8.2,7.4,11.0,7,0.98,1.0051,0.13,0.23,0.0,6.1,3.9,10.6,8,0.86,1.0086,0.12,0.27,0.0,7.7,6.8,9.1,6,0.94,1.0173,0.56,0.13,3.2,2.1,-1.3,5.5,8,0.92,1.0062,0.17,1.34,0.4,8.6,7.5,9.9,7,0.86,1.0254,0.46,0.0,0.9,9.8,7.4,12.2,6,0.72,1.018,0.61,0.30,5.1,6.2,4.2,10.2,8,0.62,1.0139,0.04,0.00,0.0,3.4,2.8,4.9,6,0.97,1.0292,0.21,0.61,0.0,-9.5,-10.5,-8.5,5,0.62,1.0114,0.05,0.06,0.0,4.0,3.0,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,19600103,1,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,0.77,1.0179,0.67,0.00,3.5,3.1,-0.5,6.4,4,0.67,1.017,0.30,0

In [6]:
df.shape

(22950, 137)

In [7]:
df1.head()

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
df1.shape

(22950, 16)

In [9]:
df.shape

(22950, 137)

### 2. Data Wrangling

In [10]:
# Drop unnecessary columns

df.drop(['DATE', 'MONTH'], axis=1, inplace=True)

In [11]:
df.shape

(22950, 135)

In [12]:
df1.drop(columns = 'DATE', inplace = True)

In [13]:
df1.shape

(22950, 15)

# 3. Reshaping for modeling
a. To ensure the layers can be fed to the deep learning model correctly.
    
b. Split the observations (X) into 15 groups of 9 types of observations.
c. Labels (y) should also be in 15 groups (it doesn’t need to be transformed or reshaped).

d. The final shapes should be X = (22950, 15, 9) and y = (22950, 15).

In [14]:
X = df

In [15]:
X.shape

(22950, 135)

In [16]:
# Turn X and y from a df to arrays

X = np.array(X)
y = np.array(df1)

In [17]:
X = X.reshape(-1,15,9)

In [18]:
# Verify shape

X.shape

(22950, 15, 9)

In [19]:
# Use argmax to transform y

bayes_y =  np.argmax(y, axis = 1)
bayes_y 

array([0, 0, 0, ..., 0, 0, 0])

In [20]:
bayes_y .shape

(22950,)

In [21]:
# Check bayes_y layout

from sklearn.utils.multiclass import type_of_target
type_of_target(bayes_y )

'multiclass'

# 4. Data Split

In [27]:
# Split data into train and test sets

X_train, X_test, Y_train, Y_test = train_test_split(X,bayes_y,random_state = 42)

In [28]:
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(17212, 15, 9) (17212,)
(5738, 15, 9) (5738,)


# 5. Bayesian Hyper-parameter Optimisation

In [35]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 # Number of weather statio
score_acc = make_scorer(accuracy_score)
# Create function
def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']

    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)

    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))

        if normalization > 0.5:
           model.add(BatchNormalization())
        for i in range(layers1):
           model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
           model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
           model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, Y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [36]:
import time
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000),
    'epochs':(20, 100),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) #25
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/32
15/15 - 5s - loss: 2.7068 - accuracy: 0.5979 - 5s/epoch - 336ms/step
Epoch 2/32
15/15 - 4s - loss: 2.7004 - accuracy: 0.6440 - 4s/epoch - 239ms/step
Epoch 3/32
15/15 - 3s - loss: 2.6971 - accuracy: 0.6440 - 3s/epoch - 230ms/step
Epoch 4/32
15/15 - 4s - loss: 2.6942 - accuracy: 0.6440 - 4s/epoch - 238ms/step
Epoch 5/32
15/15 - 3s - loss: 2.6917 - accuracy: 0.6440 - 3s/epoch - 220ms/step
Epoch 6/32
15/15 - 3s - loss: 2.6895 - accuracy: 0.6440 - 3s/epoch - 226ms/step
Epoch 7/32
15/15 - 2s - loss: 2.6873 - accuracy: 0.6440 - 2s/epoch - 161ms/step
Epoch 8/32
15/15 - 3s - loss: 2.6854 - accuracy: 0.6440 - 3s/epoch - 223ms/step
Epoch 9/32
15/15 - 3

ValueError: Input y contains NaN.

In [37]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]

optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

{'activation': 'softsign',
 'batch_size': 460,
 'dropout': 0.7296061783380641,
 'dropout_rate': 0.19126724140656393,
 'epochs': 91,
 'kernel': 1.9444298503238986,
 'layers1': 1,
 'layers2': 2,
 'learning_rate': 0.7631771981307285,
 'neurons': 61,
 'normalization': 0.770967179954561,
 'optimizer': <keras.optimizers.optimizer_v2.adadelta.Adadelta at 0x7525bab455a0>}

# 6. Creating Keras Model with Optimised Hyper-parameters

In [39]:
neurons = 61
kernel = round(1.9444298503238986)
activation = 'softsign'
optimizer = Adadelta(learning_rate=0.770967179954561)
batch_size = 460
dropout = 0.7296061783380641
dropout_rate = 0.19126724140656393
normalization = 0.770967179954561
epochs = 91
layers1 = 1
layers2 = 2

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))

if normalization > 0.5:
   model.add(BatchNormalization())
for i in range(layers1):
   model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
if dropout > 0.5:
   model.add(Dropout(dropout_rate, seed=123))
for i in range(layers2):
   model.add(Dense(neurons, activation=activation))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [40]:
model.summary()

Model: "sequential_137"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_132 (Conv1D)         (None, 14, 61)            1159      
                                                                 
 batch_normalization_51 (Bat  (None, 14, 61)           244       
 chNormalization)                                                
                                                                 
 dense_670 (Dense)           (None, 14, 61)            3782      
                                                                 
 dropout_77 (Dropout)        (None, 14, 61)            0         
                                                                 
 dense_671 (Dense)           (None, 14, 61)            3782      
                                                                 
 dense_672 (Dense)           (None, 14, 61)            3782      
                                                    

In [45]:
from tensorflow.keras.utils import to_categorical
# Put the y_test set back into a one-hot configuration

Y_train_one_hot = to_categorical(Y_train, num_classes=15)

In [46]:
# Check shapes

print(f'X_train shape: {X_train.shape}')
print(f'y_train_one_hot shape: {Y_train_one_hot.shape}')

X_train shape: (17212, 15, 9)
y_train_one_hot shape: (17212, 15)


In [47]:
model.fit(X_train, Y_train_one_hot, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/91
38/38 - 9s - loss: 1.3147 - accuracy: 0.6350 - 9s/epoch - 226ms/step
Epoch 2/91
38/38 - 6s - loss: 0.9090 - accuracy: 0.6896 - 6s/epoch - 162ms/step
Epoch 3/91
38/38 - 7s - loss: 0.8239 - accuracy: 0.7100 - 7s/epoch - 184ms/step
Epoch 4/91
38/38 - 5s - loss: 0.7720 - accuracy: 0.7283 - 5s/epoch - 120ms/step
Epoch 5/91
38/38 - 7s - loss: 0.7227 - accuracy: 0.7473 - 7s/epoch - 177ms/step
Epoch 6/91
38/38 - 7s - loss: 0.6801 - accuracy: 0.7659 - 7s/epoch - 179ms/step
Epoch 7/91
38/38 - 8s - loss: 0.6450 - accuracy: 0.7781 - 8s/epoch - 206ms/step
Epoch 8/91
38/38 - 7s - loss: 0.6024 - accuracy: 0.7936 - 7s/epoch - 182ms/step
Epoch 9/91
38/38 - 8s - loss: 0.5813 - accuracy: 0.7998 - 8s/epoch - 222ms/step
Epoch 10/91
38/38 - 7s - loss: 0.5529 - accuracy: 0.8087 - 7s/epoch - 196ms/step
Epoch 11/91
38/38 - 7s - loss: 0.5264 - accuracy: 0.8190 - 7s/epoch - 174ms/step
Epoch 12/91
38/38 - 8s - loss: 0.5210 - accuracy: 0.8218 - 8s/epoch - 207ms/step
Epoch 13/91
38/38 - 4s - loss: 0.4965

# 7. Creating Confusion Matrix

In [48]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [49]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [52]:
from tensorflow.keras.utils import to_categorical
# Put the y_test set back into a one-hot configuration

Y_test_one_hot = to_categorical(Y_test, num_classes=15)

In [53]:
# Evaluate

print(confusion_matrix(Y_test_one_hot, model.predict(X_test)))

180/180 [==============================] - 1s 3ms/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL        3644        22         6       1           0         2       0   
BELGRADE       84       999         4       0           0         1       0   
BUDAPEST       17        14       173       4           0         1       0   
DEBILT          3         3        10      63           2         1       0   
DUSSELDORF      4         1         0       3           9        10       0   
HEATHROW        7         1         0       1           2        66       0   
KASSEL          1         0         0       0           0         0       7   
LJUBLJANA       7         3         1       0           1         0       0   
MAASTRICHT      4         0         0       0           0         1       0   
MADRID         33         3         4       0           0         2       0 